# Infer Pooled Model Parameters from Individuals in Lung Cancer Control Group

## Show control group data

In [1]:
import erlotinib as erlo


# Get data
data = erlo.DataLibrary().lung_cancer_control_group()

# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, id_key='#ID', time_key='TIME in day', biom_key='TUMOUR VOLUME in cm^3')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in 8 mice with patient-derived lung cancer implants.

## Find MAP estimates for pooled model parameters

In [2]:
import pints


# Define model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch_reparametrised()
model = erlo.PharmacodynamicModel(path)
model.set_parameter_names(names={
        'myokit.tumour_volume': 'Tumour volume in cm^3',
        'myokit.critical_volume': 'Critical volume in cm^3',
        'myokit.drug_concentration': 'Drug concentration in mg/L',
        'myokit.kappa': 'Potency in L/mg/day',
        'myokit.lambda': 'Exponential growth rate in 1/day'})

# Create model
problem = erlo.ProblemModellingController(
    data, id_key='#ID', time_key='TIME in day', biom_keys=['TUMOUR VOLUME in cm^3'])
problem.set_mechanistic_model(model)
problem.set_error_model(
    error_models=[pints.ConstantAndMultiplicativeGaussianLogLikelihood])
problem.fix_parameters(name_value_dict=dict({
    'Drug concentration in mg/L': 0,
    'Potency in L/mg/day': 0,
    'Noise param 2': 1}))
population_models = [erlo.PooledModel] * 5
problem.set_population_model(population_models)

# Create posterior
log_prior_tumour_volume = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_critical_volume = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_lambda = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=3)
log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=3)
problem.set_log_prior(log_priors=[
    log_prior_tumour_volume, 
    log_prior_critical_volume,
    log_prior_lambda,
    log_prior_sigma_base,
    log_prior_sigma_rel])
log_posteriors = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posteriors)
opt.set_transform(transform=pints.LogTransformation(n_parameters=5))
map_estimates = opt.run(show_id_progress_bar=True)

### Visualise optimisation results

In [3]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 2:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value across a number of optimisation runs.

## Find posterior probability distribution

In [4]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posteriors)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=5))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

In [5]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 3:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples.